In [1]:
import requests
from bs4 import BeautifulSoup , SoupStrainer
import traceback
import pandas as pd
import calendar
import datetime
import time
import winsound
import os
import re
import urllib.parse
import urllib
from urllib.request import Request, urlopen
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.desired_capabilities import DesiredCapabilities
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import Select
from selenium.webdriver.firefox.options import Options
from selenium.webdriver.firefox.firefox_binary import FirefoxBinary
from selenium.webdriver.common.action_chains import ActionChains

from fake_useragent import UserAgent


frequency = 2500  # Set Frequency To 2500 Hertz
duration = 1000  # Set Duration To 1000 ms == 1 second


ua = UserAgent()
username = os.getenv("USERNAME")
binary = FirefoxBinary(r'C:\Program Files\Mozilla Firefox\firefox.exe')
fp = (r'C:\Users\Yash Gupta\AppData\Roaming\Mozilla\Firefox\Profiles\q7imc2hk.yashscrape')
opts = Options()
opts.profile = fp
firefox_capabilities = DesiredCapabilities.FIREFOX
firefox_capabilities["pageLoadStrategy"] = "normal"
opts.add_argument(f'user-agent=ua.chrome')

headers = {
        "User-Agent": "Mozilla/5.0 AppleWebKit/537.36 (KHTML, like Gecko; compatible; Googlebot/2.1; +http://www.google.com/bot.html) Chrome/W.X.Y.Z Safari/537.36",
        "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,*/*;q=0.8",
        "Accept-Language": "en-US,en;q=0.5",
        "Accept-Encoding": "gzip, deflate",
        "Connection": "keep-alive",
        "Upgrade-Insecure-Requests": "1",
        "Sec-Fetch-Dest": "document",
        "Sec-Fetch-Mode": "navigate",
        "Sec-Fetch-Site": "none",
        "Sec-Fetch-User": "?1",
        "Cache-Control": "max-age=0",
        "Referer": "https://www.google.com/"
    }
# headers = {
#         "User-Agent": ua.random,
#     }

C:\Users\Yash Gupta\AppData\Local\Temp\ipykernel_16476\4109902469.py:38: DeprecationWarning: Setting a profile has been deprecated. Please use the set_preference and install_addons methods
  opts.profile = fp


In [2]:
df_list = pd.read_csv("seller_detail.csv")

In [3]:
df_list

,seller_name,seller_url,seller_feedback,seller_items_sold,seller_followers,seller_search_items,seller_about_location,seller_member_since,seller_about_name
0,**Australian Commonwealth Coins**,https://www.ebay.com.au/str/ambler57,100% positive Feedback,22K items sold,479 followers,948 items,Australia,16 September 2009,ambler57
1,0ceanicshark,https://www.ebay.com.au/str/0ceanicshark,100% positive Feedback,19K items sold,284 followers,415 items,Australia,01 May 2005,oceanic_shark
2,1000 Things Australia,https://www.ebay.com.au/str/1000thingsaustralia,99.4% positive Feedback,9.3K items sold,374 followers,"1,159 items",Australia,26 March 2004,1000thingsaus
3,1000GiftsOnline,https://www.ebay.com.au/str/1000giftsonline,97.5% positive Feedback,91K items sold,679 followers,"5,087 items",Australia,30 September 2013,1000giftsonline
4,101 Decals,https://www.ebay.com.au/str/101decals,99.8% positive Feedback,26K items sold,433 followers,675 items,Australia,26 July 2008,101decals
...,...,...,...,...,...,...,...,...,...
4402,zphone australia,https://www.ebay.com.au/str/zphoneaustralia,100% positive Feedback,5.4K items sold,228 followers,20 items,Australia,09 September 2014,zphone_au
4403,zsjladyshop,https://www.ebay.com.au/str/zsjladyshop,97.2% positive Feedback,1.8M items sold,7.1K followers,556 items,China,01 December 2009,zsjladyshop
4404,zssz201882017,https://www.ebay.com.au/str/zssz201882017,98.7% positive Feedback,14K items sold,323 followers,845 items,China,09 November 2017,zssz201882017
4405,zulu99_australia,https://www.ebay.com.au/str/zulu99australia,100% positive Feedback,21K items sold,529 followers,113 items,Australia,09 November 2001,zulu_99


In [6]:
from googleapiclient import discovery
import json
#pip install -U oauth2client
from oauth2client.client import GoogleCredentials
from google.oauth2 import service_account

#pip install pandas-gbq -U
from google.cloud import bigquery
from google.cloud import secretmanager
from google.cloud import resourcemanager_v3

#os.environ['http_proxy'] = 'http://3.24.58.156:3128'
#os.environ['https_proxy'] = 'https://3.24.58.156:3128'
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = os.getcwd() + "\\application_default_credentials.json"
credentials = GoogleCredentials.get_application_default()
sm_client = secretmanager.SecretManagerServiceClient()
sa_key = json.loads(sm_client.access_secret_version(request={"name":'projects/1060770463555/secrets/space-usage/versions/1'}).payload.data.decode("UTF-8"))
credentials = service_account.Credentials.from_service_account_info(sa_key)
#Setup Application Default Credentials -- https://cloud.google.com/docs/authentication/client-libraries#python
client = bigquery.Client(project='gcp-wow-rwds-ai-bi-prod', credentials=credentials)
dept_table_id = "gcp-wow-rwds-ai-bi-prod.market_place.ebay_scrape_sellers"

job_config = bigquery.LoadJobConfig(
         autodetect = True,
     )
job_config.write_disposition = 'WRITE_TRUNCATE'
job_dept = client.load_table_from_dataframe(df_list, dept_table_id, job_config=job_config)
job_dept.result() 


C:\Users\Yash Gupta\AppData\Local\Programs\Python\Python311\Lib\site-packages\google\auth\_default.py:78: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. You might receive a "quota exceeded" or "API not enabled" error. See the following page for troubleshooting: https://cloud.google.com/docs/authentication/adc-troubleshooting/user-creds. 
  warnings.warn(_CLOUD_SDK_CREDENTIALS_WARNING)


LoadJob<project=gcp-wow-rwds-ai-bi-prod, location=US, id=c218f7e6-a69d-4916-a718-a31fc9ff79f1>